# Preprocessing
- Drop redundancies
- Handing missing values
- Encoding

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
dataset = pd.read_excel('pharma.xlsx')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791679 entries, 0 to 791678
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Mã đơn hàng    721231 non-null  object        
 1   Mã KH          721231 non-null  object        
 2   Tên KH         721160 non-null  object        
 3   Tỉnh           709286 non-null  object        
 4   Quận/huyện     669461 non-null  object        
 5   Xã/phường      700076 non-null  object        
 6   Ngày           721231 non-null  datetime64[ns]
 7   SDT            721231 non-null  float64       
 8   Tên nhân viên  721231 non-null  object        
 9   Chi nhánh      721231 non-null  object        
 10  Mã thuốc       721231 non-null  object        
 11  Tên thuốc      721231 non-null  object        
 12  Đơn vị         721231 non-null  object        
 13  Số lượng       721231 non-null  float64       
 14  Đơn giá        721231 non-null  float64       
dtype

In [4]:
dataset.head()

,Mã đơn hàng,Mã KH,Tên KH,Tỉnh,Quận/huyện,Xã/phường,Ngày,SDT,Tên nhân viên,Chi nhánh,Mã thuốc,Tên thuốc,Đơn vị,Số lượng,Đơn giá
0,0c705e68-cbff-4c25-b860-881c51d7b291,N00146,Nhà thuốc Bệnh viện Nam Thăng Long,Thành phố Hà Nội,Quận Bắc Từ Liêm,Phường Xuân Đỉnh,2018-10-04,983855018.0,Phạm Thu Trang,Chi nhánh Hà Nội,G00817,Golistin-enema for children,LO,50.0,26667.0
1,0c705e68-cbff-4c25-b860-881c51d7b291,N00146,Nhà thuốc Bệnh viện Nam Thăng Long,Thành phố Hà Nội,Quận Bắc Từ Liêm,Phường Xuân Đỉnh,2018-10-04,983855018.0,Phạm Thu Trang,Chi nhánh Hà Nội,G00821,Golistin-enema,LO,30.0,35238.0
2,0c705e68-cbff-4c25-b860-881c51d7b291,N00146,Nhà thuốc Bệnh viện Nam Thăng Long,Thành phố Hà Nội,Quận Bắc Từ Liêm,Phường Xuân Đỉnh,2018-10-04,983855018.0,Phạm Thu Trang,Chi nhánh Hà Nội,P01191,Phenhalal,ONG,200.0,3950.0
3,196999A6-37EE-405C-8B9E-33E48A0455DC,P00641,Phòng khám đa khoa Hoàng Dũng - Quảng Trị,Tỉnh Quảng Trị,Thành phố Đông Hà,Phường 1,2018-10-04,977519929.0,Lưu Hữu Công,Chi nhánh Đà Nẵng,Z00209,Zentokid - H10,ONG,400.0,6500.0
4,5a1d6b35-5755-44d7-ad79-14f60dd4f0ef,B00670,Bệnh Viện Đa Khoa Huyện Tuyên Hóa ( Nhà Thuốc),Tỉnh Quảng Bình,NaN,Thị Trấn Đồng Lê -Huyện Tuyên Hoá -Tỉnh Quảng ...,2018-10-04,911561101.0,Ngô Minh Anh Văn,Chi nhánh Đà Nẵng,H00852,Hemotocin,LO,10.0,235700.0


In [5]:
columns = dataset.columns
columns

Index(['Mã đơn hàng', 'Mã KH', 'Tên KH', 'Tỉnh', 'Quận/huyện', 'Xã/phường',
       'Ngày ', 'SDT', 'Tên nhân viên', 'Chi nhánh', 'Mã thuốc', 'Tên thuốc',
       'Đơn vị', 'Số lượng', 'Đơn giá'],
      dtype='object')

In [6]:
number_unique = {}
for column in columns:
    number_unique[column] = dataset[column].nunique()
    print(f'{column}: {dataset[column].nunique()}')

Mã đơn hàng: 341939
Mã KH: 61808
Tên KH: 49183
Tỉnh: 3800
Quận/huyện: 3707
Xã/phường: 12389
Ngày : 1351
SDT: 1846
Tên nhân viên: 1714
Chi nhánh: 4
Mã thuốc: 341
Tên thuốc: 457
Đơn vị: 10
Số lượng: 1453
Đơn giá: 776


In [7]:
top = dataset['Tỉnh'].value_counts()
top.head(20)

Thành phố Hà Nội         95736
Tỉnh Thanh Hóa           34058
Thành phố Hồ Chí Minh    33184
Tỉnh Nghệ An             30783
Tỉnh Thái Bình           29895
Tỉnh Thái Nguyên         27307
Thành phố Hải Phòng      26176
Thành phố Đà Nẵng        25161
Tỉnh Nam Định            24963
Tỉnh Quảng Ninh          17997
Tỉnh Hải Dương           16911
Tỉnh Tuyên Quang         16837
Tỉnh Vĩnh Phúc           16527
Tỉnh Bắc Ninh            16123
Tỉnh Hưng Yên            15117
Tỉnh Hà Nam              13976
Tỉnh Ninh Bình           13364
Tỉnh Bắc Giang           13168
Tỉnh Quảng Trị           12569
Tỉnh Phú Thọ             10934
Name: Tỉnh, dtype: int64

## Drop redundancies
- Drop columns with vast categories (numbers of unique)

### Case 1
- mã đơn hàng, mã kh, tên kh, tên nhân viên: drop vì
    - nunique lớn (nhiều thể loại)
    - 
- Dơn giá không liên quan vì = số lượng x giá (giá có thể thay đổi nhưng drop để đơn giản trong TH này)
- Tên thuốc dễ nhập sai -> giữ mã thuốc, bỏ tên thuốc, đơn vị
- địa điểm: chi nhánh, Tỉnh, quận/huyện, xã/phường -> chọn chi nhánh (drop còn lại)

In [8]:
drop_list = ['Mã đơn hàng', 'Mã KH', 'Tên KH', 'Xã/phường', 'Quận/huyện', 'Tên nhân viên', 'Tên thuốc', 'Đơn vị', 'Đơn giá', 'Chi nhánh' ]
df1 = dataset.drop(drop_list, axis=1)
df1.head()

,Tỉnh,Ngày,SDT,Mã thuốc,Số lượng
0,Thành phố Hà Nội,2018-10-04,983855018.0,G00817,50.0
1,Thành phố Hà Nội,2018-10-04,983855018.0,G00821,30.0
2,Thành phố Hà Nội,2018-10-04,983855018.0,P01191,200.0
3,Tỉnh Quảng Trị,2018-10-04,977519929.0,Z00209,400.0
4,Tỉnh Quảng Bình,2018-10-04,911561101.0,H00852,10.0


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791679 entries, 0 to 791678
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Tỉnh      709286 non-null  object        
 1   Ngày      721231 non-null  datetime64[ns]
 2   SDT       721231 non-null  float64       
 3   Mã thuốc  721231 non-null  object        
 4   Số lượng  721231 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 30.2+ MB


In [10]:
# Step 1: Get the value counts
value_counts = df1['Tỉnh'].value_counts()
value_counts.head(7)

Thành phố Hà Nội         95736
Tỉnh Thanh Hóa           34058
Thành phố Hồ Chí Minh    33184
Tỉnh Nghệ An             30783
Tỉnh Thái Bình           29895
Tỉnh Thái Nguyên         27307
Thành phố Hải Phòng      26176
Name: Tỉnh, dtype: int64

### Choose top 7 popular city:
- Hà Nội, Thanh Hóa, Nghệ An, Thái Bình, Tp Hồ Chí Minh, Thái Nguyên, Thành phố Hải Phòng

In [11]:
# Step 2: Get the top 7 values
top_values = value_counts.nlargest(7).index.tolist()

# Step 3: Create a boolean mask
mask = df1['Tỉnh'].isin(top_values)

# Step 4: Select the rows that contain the top 7 values
df_top7 = df1.loc[mask]
df_top7

,Tỉnh,Ngày,SDT,Mã thuốc,Số lượng
0,Thành phố Hà Nội,2018-10-04,983855018.0,G00817,50.0
1,Thành phố Hà Nội,2018-10-04,983855018.0,G00821,30.0
2,Thành phố Hà Nội,2018-10-04,983855018.0,P01191,200.0
14,Thành phố Hà Nội,2018-10-10,903251248.0,G00898,5.0
15,Tỉnh Thanh Hóa,2018-10-13,912208899.0,D01057,4000.0
...,...,...,...,...,...
791673,Thành phố Hà Nội,2022-07-11,969512158.0,L01021,1.0
791674,Thành phố Hà Nội,2022-07-11,969512158.0,N01080,2.0
791675,Thành phố Hà Nội,2022-07-11,969512158.0,N01084,2.0
791677,Thành phố Hà Nội,2022-07-11,969512158.0,P01364,35.0


In [12]:
df_top7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277139 entries, 0 to 791678
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Tỉnh      277139 non-null  object        
 1   Ngày      277139 non-null  datetime64[ns]
 2   SDT       277139 non-null  float64       
 3   Mã thuốc  277139 non-null  object        
 4   Số lượng  277139 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 12.7+ MB


### Select top 8 popular drugs
- Step 1: Choose top 50 popular
- Step 2: Group by ATC code
- Step 3: Select essential group then use their drugs

In [13]:
drug_50 = df_top7['Mã thuốc'].value_counts()
drug_50.head(50)

N01080    9401
M01225    9291
Z00200    9031
G00821    8476
P01481    8342
M01090    8195
P01364    7201
G00898    7069
P01146    6874
Z00209    5756
Z00226    5074
F00550    5025
B01373    4776
G00869    4732
C01842    4712
B01166    4044
L01012    3914
Z00197    3549
N01041    3498
A01414    3263
D01304    3122
R00365    2865
H01921    2819
T01864    2723
D01375    2659
N01084    2624
P01478    2591
D01349    2574
A01303    2272
T01855    2184
S01447    2175
P01299    2148
N00845    2003
L00993    1993
T01701    1894
L00854    1820
N01165    1801
Z00219    1797
D01183    1772
L00801    1766
D01355    1721
P01336    1580
H00852    1541
F00482    1523
D01063    1509
M01656    1445
G00817    1439
G00951    1435
A01338    1421
M01076    1407
Name: Mã thuốc, dtype: int64